# Quick Start

In this notebook, we show how you can inject any functions to specific event during the MAS execution, so you can perform more type of attacks instead of just prompt attack,

In the previous notebooks, we showed how run agent, MAS, and task suite without injected functions,

now lets see how can you do it, we begin with how to inject functions to the run execution of an single agent


Again, make sure you set the API keys correclty before proceed and please replace the models to the ones you have access to if necessary

In [1]:
import os

#os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"
#os.environ["GEMINI_API_KEY"] = "your_gemini_api_key_here"
#os.environ["DEEPSEEK_API_KEY"] = "your_deepseek_api_key_here"
#os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key_here"
# Additional api keys can be set in a similar manner

In [2]:
# Import necessary classes

from mav.MAS.agents import (
    Agent,
    Runner,
    RunResult
)

# Load environment variables
from dotenv import load_dotenv
load_dotenv()


# In case src is not in the path
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "src"))

In [3]:
def get_weather(city: str) -> str:
    """
    Use this function to get weather information for a given city.
    Args:
        city (str): The name of the city to get the weather for.
    """
    return f"The weather in {city} is sunny with a high of 75°F."

weather_agent = Agent(
    name="WeatherAgent",
    model="openai/gpt-5-mini",
    instructions="You are a weather assistant. Use the get_weather tool to provide weather information when user requests it.",
    tools=[get_weather],
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096
    }
)

### Pass Attack Hooks to Agent Runner

You can pass any number of functions to the agent `Runner.run()` as a list, as long as these functions take exactly three named parameters:

1. `event`: This is used to tell the function which specifc event that the agent runner or the MAS runner is at, for examples for the agent runner, we have a pre-defined list of events:

    1. run_start: This event denotes the step right before we go into the ReACT agent loop for an agent
    2. before_model_call: This event denotes the step before we make the LLM call at each iteration
    3. after_model_call:  This event denotes the step after we received the LLM response from our call
    4. after_tool_calls: This event denotes the step after the loop finished the tool calls
    5. run_end: This event denotes the step right before we end the runner and return the RunResult object

2. `agent_run_state`: An instance of the `Agent_Run_State` contains the following objects:

    1. agent: The agent object, for example, you can know which agent is being executed
    2. context: The context (task environment) you passed to the runner
    3. session: Either the session you created or the temporal in memory session the runner created if you do not pass one, you can modify the input items 
    4. iteration: The current iteration (0-indexed)

3. `MAS_run_state`: An dict contains the state of the parent MAS runner execution, this will be provided automatically if the agent is called by an upper level MAS, where you can access the MAS run state if needed, will discuss this more later

For example, I will create a simple function that simply prints out the input to the agent at the event `run_start`

In [5]:
from mav.MAS.agents.run import Agent_Run_State
from typing import Any

async def print_input(event, agent_run_state: Agent_Run_State, MAS_run_state: dict[str, Any] | None):
    if event == "run_start":
        input_items = await agent_run_state.session.get_items()
        print(f"Agent {agent_run_state.agent.name} is starting a run with input: {input_items}")

In [6]:
result = await Runner.run(
    agent=weather_agent,
    input="What's the weather like in New York City today?",
    attack_hooks=[print_input]
)

Agent WeatherAgent is starting a run with input: [{'role': 'user', 'content': "What's the weather like in New York City today?"}]


For how each event is actually defined, please take a look at our agent Runner implementation

### Pass Attack Hooks to MAS Runner

Next, lets see how can you pass attack hooks to the MAS Runner, unlike the agent runner, where the flow is determined (ReACT loop), MAS can have many kinds of workflow, hence we leave the event defintion and the MAS_run_state (a dict) to the user when brining their own MAS workflow, for our pre-built MAS, i.e., planner-executor, we have the following events:

1. mas_run_start: start of the MAS runner
2. planner_turn_start: the step before the planner start running at each iteration
3. executor_turn_start: the step before the executor start running at each iteration
4. mas_run_end: the end of the MAS runner


For the MAS_run_state, we define it to be a dict with the following keys:

1. "iteration": iteration,
2. "planner_input": the input to the planner at each iteration,
3. "executor_input": the input to the executor at each iteration,
4. "planner_memory": the planner session memory object, if specified otherwise None,
5. "executor_memory": the executor session memory object, if specified otherwise None,
6. "context": The context (task environment) object passed the agent runner for both planner and executor

Again, we will upgrade the print function a little bit to show case:

In [ ]:
async def print_input(event, agent_run_state: Agent_Run_State, MAS_run_state: dict[str, Any] | None):
    if event == "run_start":
        input_items = await agent_run_state.session.get_items()
        mas_iteration = MAS_run_state.get("iteration") if MAS_run_state else None
        print(f"Agent {agent_run_state.agent.name} is starting a run with input: {input_items} at MAS iteration: {mas_iteration}")
    elif event == "mas_run_start":
        print(f"MAS is starting a run with planner input: {MAS_run_state.get('planner_input')}")

In [9]:
from mav.MAS import MultiAgentSystem, MASRunResult


exectutor_agent = Agent(
    name="ExecutorAgent",
    model="openai/gpt-5-mini",
    instructions="""You are an executor agent that can use multiple tools to complete tasks you are given and return an concise final answer.""",
    tools=[
        get_weather,
    ],
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096
    }
)

# Define the planner agent that will create plans for the executor agent
from pydantic import BaseModel
from typing import Literal
from openai.lib._parsing._responses import type_to_text_format_param

# Define the response format for the planner agent
class PlannerResponseFormat(BaseModel):
    plan: str
    final_answer: str
    status: Literal["in_progress", "task_complete", "failed"]

# Create the planner agent
planner_agent = Agent(
    name="PlannerAgent",
    model="openai/gpt-5-mini",
    instructions="""You are a planner agent that can provide a step-by-step plan to complete user tasks, your job is to provide a complete plan that can be executed by another agent,
    The executor agent is not as smart as you, so make sure to provide detailed stesps and all necessary context, as the executor agent will simply follow your plan to complete the task.
    The executor agent has access to two tools: get_weather and calculate.
    You are working in a loop with the executor agent, where you provide a plan, the executor executes it and returns the result, and you provide the next plan based on the result, until the task is complete.
    When you believe the task is complete, in your final response, please specify status as 'task_complete', before that please always specify status as 'in_progress'.
    When you are done, please provide a concise final answer in the final_answer field, otherwise leave it blank.
    When you are done, the plan field can be left blank.""",
    model_settings={
        "reasoning": {"effort": "medium"},
        "text": {"format": type_to_text_format_param(PlannerResponseFormat)},
        "max_output_tokens": 8192
    }
)

# Define the multi-agent system with the planner and executor agents
mas = MultiAgentSystem(
    agents=[planner_agent, exectutor_agent],
    MAS_runner="planner_executor"
)

# The termination condition to stop the planner-executor loop when the planner indicates task completion
from mav.MAS.terminations import PlannerExecutorMessageTerminiation
mas_termination_condition = PlannerExecutorMessageTerminiation(
    termination_message="task_complete"
)

In [ ]:
# Run the multi-agent system with a complex user request
mas_run_result: MASRunResult = await mas.query(
    # required parameters for the query method
    input="What is the weather in New York and also what is 11+11x2?",
    context=None, 
    attack_hooks=[print_input],
    # planner-executor specific parameters
    enable_planner_memory=True,
    enable_executor_memory=False,
    shared_memory=False,
    endpoint_planner="responses",
    endpoint_executor="responses",
    max_planner_iterations=5,
    max_executor_iterations=5,
    max_iterations=3,
    termination_condition=mas_termination_condition
)